# Beans dataset experiment

In this notebook we will try to study the relationship between a given subset $X$ of a dataset $Z$ and the whole dataset. The steps will be the following:
1. Load the dataset and normalize its values.
2. Explore the dataset using UMAP.
3. Choose a random subset of the dataset of a given size.
4. Train a MLP on the random subset as a train set and use the rest as test.
5. Study the partial matching between the persistence diagrams for the qualitative study

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import scipy.spatial.distance as dist
from sklearn import datasets
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import gudhi
import matplotlib.pyplot as plt
import matplotlib as mpl
from ucimlrepo import fetch_ucirepo 
from sklearn.preprocessing import LabelEncoder

import scipy.spatial.distance as dist
import itertools
from scipy.sparse.csgraph import minimum_spanning_tree

import tdqual.topological_data_quality_0 as tdqual

import warnings
warnings.filterwarnings("ignore")

plots_dir = "beans_images/"

dry_bean = fetch_ucirepo(id=602) 
Z=dry_bean["data"]
Z = preprocessing.normalize(dry_bean.data.features) 
y=dry_bean.data.targets 
le = LabelEncoder()
le.fit(y)
labels = le.transform(y)


def subsetZ(Z,labels,size):#,noise_id, size_noise):
    ids = np.random.choice(len(Z), replace=False, size=size)
    X=Z[ids]
    y=labels[ids]
    #s=np.shape(S[y==noise_id])
    #noise = np.random.normal(0,size_noise, s)
    #S[y==noise_id]=S[y==noise_id]+noise
    return X, y, ids

## Choosing $X\subseteq Z$

In [ ]:
print("Size of each class on input data:")
for i in set(labels):
    print("Class ",i,": ",np.sum(labels==i)," proportion: ", np.sum(labels==i)/len(labels))
size=3000
n_classes= len(set(labels))
X, y, X_indices = subsetZ(Z,labels,size)
print("Size of each class on train data:")
for i in set(labels):
    print("Class ",i,": ",np.sum(y==i)," proportion: ", np.sum(y==i)/len(y))

### UMAP embedding

In [ ]:
import umap.umap_ as umap
reducer = umap.UMAP(n_neighbors=100,n_components=2)
embedding = reducer.fit_transform(X)
embedding.shape

In [ ]:
for i in range(7):
    fig, (ax1, ax2) = plt.subplots(1, 2)
    ax1.plot(embedding[S_indices[y==i]][:,0],embedding[X_indices[y==i]][:,1],'.')
    ax2.plot(embedding[labels==i][:,0],embedding[labels==i][:,1],'.')
    plt.show()

## MLP training and performance measure

In [ ]:
Z_test = np.array([Z[i] for i in range(len(Z)) if i not in X_indices])
y_test = np.array([labels[i] for i in range(len(labels)) if i not in X_indices])
input_dim = np.shape(X)[1]
verbose = False
epochs = 1000
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(1024, activation="relu",use_bias=True, input_shape=(input_dim,)))
model.add(tf.keras.layers.Dense(256, activation="relu",use_bias=True, input_shape=(input_dim,)))
model.add(tf.keras.layers.Dense(128, activation="relu",use_bias=True))
model.add(tf.keras.layers.Dense(64, activation="relu",use_bias=True))
model.add(tf.keras.layers.Dense(n_classes, activation="softmax",use_bias=True))

callback = keras.callbacks.EarlyStopping(monitor='loss',
                                              patience=10)


opt  = tf.keras.optimizers.Adam(
    learning_rate=0.001
)

model.compile(optimizer=opt,loss="sparse_categorical_crossentropy",metrics=["accuracy"])
print("Training neural network...")
acc_test = []
acc_train = []
for i in range(10):
    history = model.fit(X,y,epochs=epochs, verbose = verbose,callbacks=[callback])
    acc_test.append(model.evaluate(Z_test,y_test)[1])
    acc_train.append(model.evaluate(X,y)[1])
i=np.argmax(acc_test)
print("Acc on test: ",acc_test[i])
print("Acc on train: ",acc_train[i])

In [ ]:
import numpy as np
import pandas as pd
from pretty_confusion_matrix import pp_matrix
from matplotlib import colormaps
conf_m=tf.math.confusion_matrix(y_test,[np.argmax(z) for z in model.predict(Z_test)])
# get pandas dataframe
df_cm = pd.DataFrame(conf_m, index=range(n_classes), columns=range(n_classes))
#for i in range(len(list(colormaps))):
#    print(i)
cmap = list(colormaps)[2]
pp_matrix(df_cm, cmap=cmap)

## $D(f)$ for each class

In [ ]:
for i in set(labels):
    print("Label: ",i)
    Xi = X[y==i]
    yi = y[y==i]
    Zi = Z[labels==i]
    Zi_indices = np.argwhere(labels==i).flatten() # Respect to X
    Xi_indices = X_indices[y==i] # Respect to X
    Xi_indices=[np.argwhere(Zi_indices==x).flatten()[0] for x in Xi_indices]
    
    
    Xi_compl = np.ones(Zi.shape[0], dtype="bool")
    Xi_compl[Xi_indices] = False
    Zi = np.vstack((Zi[Xi_indices], Zi[Xi_compl]))
    Xi_indices = range(len(Xi_indices))
    Xi = Zi[Xi_indices]
    filt_Xi, filt_Zi, matchingi = tdqual.compute_Mf_0(Xi,Zi)

    fig, ax = plt.subplots(figsize=(7,2.5))
    tdqual.plot_matching_0(filt_Xi, filt_Zi, matchingi, ax)
    plt.tight_layout()
    plt.show()
    plt.savefig(plots_dir + "block_matching_0_c"+str(i)+".png")

    fig, ax = plt.subplots(figsize=(3,3))
    D_fi, multiplicitiesi = tdqual.compute_matching_diagram(filt_Xi, filt_Zi, matchingi, _tol=1e-5)
    tdqual.plot_matching_diagram(D_fi, ax)
    plt.tight_layout()
    plt.show()
    plt.savefig(plots_dir + "matching_diagram_0_c"+str(i)+".png")
    
    

## $D(f)$ 

In [ ]:
X_compl = np.ones(Z.shape[0], dtype="bool")
X_compl[X_indices] = False
Z = np.vstack((Z[X_indices], Z[X_compl]))
X_indices_old = X_indices
X_indices = range(len(X_indices))
X = Z[X_indices]
filt_X, filt_Z, matching = tdqual.compute_Mf_0(X,Z)

fig, ax = plt.subplots(figsize=(7,2.5))
tdqual.plot_matching_0(filt_X, filt_Z, matching, ax)
plt.tight_layout()
plt.show()
plt.savefig(plots_dir + "block_matching_0.png")

fig, ax = plt.subplots(figsize=(3,3))
D_f, multiplicities = tdqual.compute_matching_diagram(filt_X, filt_Z, matching, _tol=1e-5)
tdqual.plot_matching_diagram(D_f, ax)
plt.tight_layout()
plt.show()
plt.savefig(plots_dir + "matching_diagram_0.png")

## Umap embedding

In [ ]:
import umap.umap_ as umap
reducer = umap.UMAP(n_neighbors=100,n_components=2)
embedding = reducer.fit_transform(X)
embedding.shape

In [ ]:
plt.scatter(embedding[:,0],embedding[:,1],c=labels)
plt.colorbar(boundaries=np.arange(8)-0.5).set_ticks(np.arange(7))

In [ ]:
for i in range(7):
    fig, (ax1, ax2) = plt.subplots(1, 2)
    ax1.plot(embedding[S_indices[y==i]][:,0],embedding[X_indices[y==i]][:,1],'.')
    ax2.plot(embedding[labels==i][:,0],embedding[labels==i][:,1],'.')
    plt.show()